### Exercise 6 - Topic models

##### Exercise 6.1

In [92]:
import os
import nltk
import numpy as np
import scipy
from numpy import matlib
import sklearn
import sklearn.decomposition
import matplotlib.pyplot as plt
import sklearn.manifold
import scipy.stats

##### 6.1a

In [93]:
#%% Getting a list of directory contents
def gettextlist(directory_path):
    directory_textfiles=[]
    directory_nontextfiles=[]
    directory_nonfiles=[]
    # Process each item in the directory
    directory_contents=os.listdir(directory_path)
    for contentitem in directory_contents:
        temp_fullpath=os.path.join(directory_path, contentitem)
        # Non-files (e.g. subdirectories) are stored separately
        if os.path.isfile(temp_fullpath)==0:
            directory_nonfiles.append(contentitem)
        else:
            # Is this a non-text file (not ending in .txt)?
            if temp_fullpath.find('.txt')==-1:
                directory_nontextfiles.append(contentitem)
            else:
                # This is a text file
                directory_textfiles.append(contentitem)
    return(directory_textfiles,directory_nontextfiles,directory_nonfiles)

In [94]:
#%% Basic file crawler
def basicfilecrawler(directory_path):
    # Store filenames read and their text content
    num_files_read=0
    crawled_filenames=[]
    mycrawled_texts=[]
    directory_contentlists=gettextlist(directory_path)
    # In this basic crawled we just process text files
    # and do not handle subdirectories
    #directory_textfiles=directory_contentlists[0]
    directory_nontextfiles=directory_contentlists[1]
    for contentitem in directory_nontextfiles:
        #print('Reading file:')
        #print(contentitem)
        # Open the file and read its contents
        temp_fullpath=os.path.join(directory_path, contentitem)
        #temp_file=open(temp_fullpath,'r',encoding='utf-8',errors='ignore')
        temp_file=open(temp_fullpath,'r')
        temp_text=temp_file.read()
        temp_file.close()
        # Store the read filename and content
        crawled_filenames.append(contentitem)
        mycrawled_texts.append(temp_text)
        num_files_read=num_files_read+1
    return(crawled_filenames, mycrawled_texts)


In [95]:
crawled_filenames, mycrawled_texts = basicfilecrawler('20news')

In [96]:
len(mycrawled_texts)

4000

In [97]:
crawled_filenames[0:5]

['100521', '101551', '101552', '101553', '101554']

##### 6.1b

In [ ]:
# Exclude header lines from each message
excludedlinemarkers=['Xref:','Path:','From:','Newsgroups:','Subject:','Summary:',
                     'Keywords:','Message-ID:','Date:','Expires:','Followup-To:','Distribution:', 
                     'Organization:','Approved:','Supersedes:','Lines:','NNTP-Posting-Host:', 
                     'References:','Sender:','In-Reply-To:','Article-I.D.:','Reply-To:', 
                     'Nntp-Posting-Host:', 'Newsgroup', 'document_id']

for k in range(len(mycrawled_texts)):
    print(k)
    templines = mycrawled_texts[k].splitlines()
    remaininglines = []
    for l in range(len(templines)):
        line_should_be_excluded = 0
        for m in range(len(excludedlinemarkers)):
            if len(templines[l]) >= len(excludedlinemarkers[m]):
                if excludedlinemarkers[m] == templines[l][0:len(excludedlinemarkers[m])]:
                    line_should_be_excluded = 1
                    break
        if line_should_be_excluded == 0:
            remaininglines.append(templines[l])
    mycrawled_texts[k] = '\n'.join(remaininglines)

In [99]:
downloaded_texts = []
# remove leading and trailing whitespaces
for k in range(len(mycrawled_texts)):
    temp_texts = mycrawled_texts[k].strip()
    temp_texts = ' '.join(temp_texts.split())
    downloaded_texts.append(temp_texts)

In [101]:
len(downloaded_texts)

4000

In [102]:
# Tokenize downloaded texts and change them to NLTK format
mydownloaded_nltk_texts = []
for k in range(len(downloaded_texts)):
    temp_tokenizedtext = nltk.word_tokenize(downloaded_texts[k])
    temp_nltktext = nltk.Text(temp_tokenizedtext)
    mydownloaded_nltk_texts.append(temp_nltktext)

In [103]:
mydownloaded_nltk_texts

[<Text: The Orioles ' pitching staff again is having...>,
 <Text: In article < C4vIr5.L3r @ shuksan.ds.boeing.com > ,...>,
 <Text: In article < 1993Apr5.135153.11132 @ wdl.loral.com > gwm...>,
 <Text: THANKS TO ALL OF YOU WHO RESPONDED TO...>,
 <Text: The subject says it all . My 1984...>,
 <Text: wolfson @ regatta.sps.mot.com ( Stephen Wolfson ) writes...>,
 <Text: : TIN [ version 1.1 PL8.8>,
 <Text: I 'm looking for a replacement radio/tape player...>,
 <Text: I am thinking about getting an Infiniti G20...>,
 <Text: In article < 1993Apr5.175719.7892 @ telxon.mis.telxon.com > joes...>,
 <Text: In article < 1993Apr5.212645.15988 @ nntpd.lkg.dec.com > kenyon...>,
 <Text: In article < 3mwF2B1w165w @ njcc.wisdom.bubble.org > ,...>,
 <Text: Do n't have a list of what 's...>,
 <Text: In article < 1993Apr3.005245.10615 @ michael.apple.com > ems...>,
 <Text: In article < 1pq29p $ 29p @ seven-up.East.Sun.COM...>,
 <Text: News-Software : VAX/VMS VNEWS 1.41 In article <...>,
 <Text: This past wi

In [104]:
# Make all downloaded texts lowercase
mydownloaded_lowercase_texts = []
for k in range(len(mydownloaded_nltk_texts)):
    temp_lowercase_text = []
    for l in range(len(mydownloaded_nltk_texts[k])):
        lowercase_word = mydownloaded_nltk_texts[k][l].lower()
        temp_lowercase_text.append(lowercase_word)
    temp_lowercasetest = nltk.Text(temp_lowercase_text)
    mydownloaded_lowercase_texts.append(temp_lowercase_text)

In [105]:
len(mydownloaded_lowercase_texts)

4000

In [106]:
# Convert a POS tag for WordNet
def tagtowordnet(postag):
    wordnettag=-1
    if postag[0]=='N':
        wordnettag='n'
    elif postag[0]=='V':
        wordnettag='v'
    elif postag[0]=='J':
        wordnettag='a'
    elif postag[0]=='R':
        wordnettag='r'
    return(wordnettag)

In [107]:
# POS tag and lemmatize the loaded texts
# Download tagger and wordnet resources if you do not have them already
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# lemmatize downloaded texts
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatizetext(nltktexttolemmatize):
    # tag the text with POS tags
    taggedtext = nltk.pos_tag(nltktexttolemmatize)
    # lemmatize each word text
    lemmatizedtext = []
    for l in range(len(taggedtext)):
        # Lemmatize a word using the WordNet converted POS tag
        wordtolemmatize = taggedtext[l][0]
        wordnettag = tagtowordnet(taggedtext[l][1])
        if wordnettag != -1:
            lemmatizedword = lemmatizer.lemmatize(wordtolemmatize, wordnettag)
        else:
            lemmatizedword = wordtolemmatize
        # store the lemmatized word
        lemmatizedtext.append(lemmatizedword)
    return(lemmatizedtext)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tunde\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tunde\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [108]:
mydownloaded_lemmatizedtexts = []
for k in range(len(mydownloaded_lowercase_texts)):
    lemmatizedtext = lemmatizetext(mydownloaded_lowercase_texts[k])
    lemmatizedtext = nltk.Text(lemmatizedtext)
    mydownloaded_lemmatizedtexts.append(lemmatizedtext)

In [109]:
mydownloaded_lemmatizedtexts[0:5]

[<Text: the oriole ' pitch staff again be have...>,
 <Text: in article < c4vir5.l3r @ shuksan.ds.boeing.com > ,...>,
 <Text: in article < 1993apr5.135153.11132 @ wdl.loral.com > gwm...>,
 <Text: thanks to all of you who respond to...>,
 <Text: the subject say it all . my 1984...>]

In [110]:
# find the vocabularies of each document
myvocabularies = []
myindices_in_vocabularies = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # get unique words and where they occur
    temptext = mydownloaded_lemmatizedtexts[k]
    unique_results = np.unique(temptext, return_inverse = True)
    unique_words = unique_results[0]
    word_indices = unique_results[1]
    # store the vocabularies and the indices
    myvocabularies.append(unique_words)
    myindices_in_vocabularies.append(word_indices)

In [111]:
len(myvocabularies)

4000

In [112]:
# unify the vocabularies
# first concatenate all vocabularies
tempvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    tempvocabulary.extend(myvocabularies[k])

# find unique words among all the vocabularies
uniqueresults = np.unique(tempvocabulary, return_inverse = True)
unifiedvocabulary = uniqueresults[0]
wordindices = uniqueresults[1]

In [113]:
len(unifiedvocabulary)

42763

In [114]:
# translate previous indices to the unified vocabulary
# must keep track where each vocabulary started in the concatenated one
vocabularystart = 0
myindices_in_unifiedvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # in order to shift word indices, we must temporarily
    # convert their data type to a numpy array
    tempindices = np.array(myindices_in_vocabularies[k])
    tempindices = tempindices + vocabularystart
    tempindices = wordindices[tempindices]
    myindices_in_unifiedvocabulary.append(tempindices)
    vocabularystart = vocabularystart + len(myvocabularies[k])

In [115]:
len(unifiedvocabulary)

42763

In [116]:
len(myindices_in_unifiedvocabulary)

4000

In [117]:
myindices_in_unifiedvocabulary[1][0:10]

array([24498, 12907, 10996, 15189, 11089, 36258, 11087,   947, 21631,
       11089], dtype=int64)

In [118]:
len(unifiedvocabulary[myindices_in_unifiedvocabulary[1]])

560

In [119]:
len(mydownloaded_lemmatizedtexts[1])

560

In [120]:
# total count of each unique word over all the downloaded documents
unifiedvocabulary_totaloccurrencecounts = np.zeros((len(unifiedvocabulary),1))

# count occurrences
for k in range(len(mydownloaded_lemmatizedtexts)):
    occurrencecounts = np.zeros((len(unifiedvocabulary), 1))
    for l in range(len(myindices_in_unifiedvocabulary[k])):
        occurrencecounts[myindices_in_unifiedvocabulary[k][l]] = (occurrencecounts[myindices_in_unifiedvocabulary[k][l]]
                                                                 + 1)
    unifiedvocabulary_totaloccurrencecounts = unifiedvocabulary_totaloccurrencecounts + occurrencecounts

In [121]:
unifiedvocabulary_totaloccurrencecounts

array([[7.079e+03],
       [2.518e+03],
       [1.224e+03],
       ...,
       [2.000e+00],
       [6.000e+00],
       [2.000e+00]])

In [122]:
len(unifiedvocabulary_totaloccurrencecounts), len(unifiedvocabulary)

(42763, 42763)

In [123]:
# top-100 words according to the largest total occurrence count
highest_totaloccurrences_indices = np.argsort(-1*unifiedvocabulary_totaloccurrencecounts, axis=0)
print(np.squeeze(unifiedvocabulary[highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(unifiedvocabulary_totaloccurrencecounts[highest_totaloccurrences_indices[0:100]]))

['>' 'the' ',' '.' '--' 'be' 'a' 'to' 'i' ')' 'in' 'and' '(' 'of' ':' '@'
 'have' 'that' 'it' 'you' '!' 'do' 'for' '?' "'s" 'on' '|' "n't" '-'
 'this' 'with' 'but' 'he' "''" 'not' '0' 'they' '...' '1' 'as' '<' '``'
 'at' 'if' 'get' 'write' 'or' 'my' '2' 'article' 'game' 'go' 'would' '#'
 'can' 'about' 'one' 'will' 'all' 'there' 'an' 'what' 'from' 'out' 'so'
 '*' 'good' 'car' 'like' 'year' 'by' 'think' 'me' 'team' 'up' 'his' '%'
 'just' 'when' '3' 'make' 'more' 'no' 'your' 'know' 'any' 'say' 'who'
 'than' 'we' '4' 'time' 'some' 'play' 'well' 'see' 'how' 'player' "'m"
 'only']
[40640. 40004. 39497. 39447. 35859. 27283. 17906. 17356. 16024. 14883.
 14093. 14011. 13692. 13179. 11444.  9901.  9497.  9222.  8767.  7162.
  7079.  7035.  7012.  6686.  6181.  6043.  5925.  5222.  4878.  4748.
  4509.  4285.  4178.  4150.  4000.  3977.  3935.  3868.  3795.  3751.
  3648.  3629.  3567.  3525.  3266.  3261.  3238.  3217.  2837.  2745.
  2726.  2692.  2684.  2518.  2408.  2392.  2371.  2306.  2274.

In [124]:
#%% Vocabulary pruning
nltkstopwords = nltk.corpus.stopwords.words('english')
pruningdecisions = np.zeros((len(unifiedvocabulary),1))
for k in range(len(unifiedvocabulary)):
    # Rule 1: check the nltk stop word list
    if (unifiedvocabulary[k] in nltkstopwords):
        pruningdecisions[k] = 1
    # Rule 2: if the word is too short
    if (len(unifiedvocabulary[k]) < 3):
        pruningdecisions[k] = 1
      # Rule 3: if the word is too long
    if (len(unifiedvocabulary[k]) > 20):
        pruningdecisions[k] = 1
    # Rule 4: if the word is in the top 1% of frequent words
    if (k in highest_totaloccurrences_indices[0:int(np.floor(len(unifiedvocabulary)*0.01))]):
        pruningdecisions[k] = 1
    # Rule 5: if the word occurs less than 4 times
    if(unifiedvocabulary_totaloccurrencecounts[k] < 4):
        pruningdecisions[k] = 1

In [125]:
print('Top-100 words after pruning the unified vocabulary:\n')
remaining_indices = np.squeeze(np.where(pruningdecisions==0)[0])
remaining_vocabulary = unifiedvocabulary[remaining_indices]
remainingvocabulary_totaloccurrencecounts = unifiedvocabulary_totaloccurrencecounts[remaining_indices]
remaining_highest_totaloccurrences_indices = np.argsort(-1*remainingvocabulary_totaloccurrencecounts, axis=0)
print(np.squeeze(remaining_vocabulary[remaining_highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(remainingvocabulary_totaloccurrencecounts[remaining_highest_totaloccurrences_indices[0:100]]))

Top-100 words after pruning the unified vocabulary:

['ago' 'cost' 'large' 'city' 'mind' 'total' 'mention' 'brave' 'man' 'ford'
 'penalty' 'final' 'anyway' 'bill' 'coach' 'idea' 'saw' 'performance'
 'beat' 'version' 'rule' 'montreal' 'hitter' 'seat' 'group' 'friend' 'ice'
 'today' 'face' 'although' 'order' 'almost' 'comment' 'minute' 'stats'
 'bos' 'hold' 'det' 'follow' 'job' 'american' 'insurance' 'month' 'smith'
 'past' 'local' 'easy' 'honda' 'tie' 'cal' 'news' 'hell' 'x-newsreader'
 'wait' 'ticket' 'bring' 'jet' 'walk' 'hope' 'helmet' 'rider' 'joe'
 'stuff' 'morris' 'van' 'left' 'compare' 'note' 'word' 'york' 'experience'
 'center' 'flyer' 'information' 'add' 'break' 'defense' 'puck' 'set' 'pen'
 'rear' 'contact' 'netcom.com' 'others' 'e-mail' 'later' 'young' 'claim'
 'late' 'whether' 'tor' 'design' 'matter' 'pit' 'life' 'pull' 'decide'
 'instead' 'open' 'difference']
[247. 246. 246. 246. 245. 244. 244. 243. 242. 241. 240. 240. 236. 232.
 231. 230. 229. 229. 228. 225. 223. 223. 223.

In [126]:
#%% Get indices of documents to remaining words
oldtopruned=[]
tempind=-1
for k in range(len(unifiedvocabulary)):
    if pruningdecisions[k]==0:
        tempind=tempind+1
        oldtopruned.append(tempind)
    else:
        oldtopruned.append(-1)

In [127]:
#%% Create pruned texts
mycrawled_prunedtexts=[]
myindices_in_prunedvocabulary=[]
for k in range(len(mydownloaded_lemmatizedtexts)):
    #print(k)
    temp_newindices=[]
    temp_newdoc=[]
    for l in range(len(mydownloaded_lemmatizedtexts[k])):
        temp_oldindex=myindices_in_unifiedvocabulary[k][l]
        temp_newindex=oldtopruned[temp_oldindex]
        if temp_newindex!=-1:
            temp_newindices.append(temp_newindex)
            temp_newdoc.append(unifiedvocabulary[temp_oldindex])
    mycrawled_prunedtexts.append(temp_newdoc)
    myindices_in_prunedvocabulary.append(temp_newindices)


In [128]:
len(mycrawled_prunedtexts[1])

85

In [129]:
len(remaining_vocabulary)

12087

##### 6.1c

In [39]:
# create count vectors
n_docs = len(mycrawled_prunedtexts) # 4000=total number of documents
n_vocab = len(remaining_vocabulary) # 12087
# matrix of term frequencies
tfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab)) # 4000x12087
# row vector of document frequencies
dfvector = scipy.sparse.lil_matrix((1, n_vocab)) #1x12087

In [40]:
n_docs, n_vocab

(4000, 12087)

In [41]:
tfmatrix.shape, dfvector.shape

((4000, 12087), (1, 12087))

In [42]:
# loop over the documents
for k in range(n_docs):
    # row vector of which words occur in this document
    temp_dfvector = scipy.sparse.lil_matrix((1, n_vocab)) # #1x12087
    # we loop over the words in each document
    for l in range(len(mycrawled_prunedtexts[k])):
        # add current word to term-frequency count and document count
        currentword = myindices_in_prunedvocabulary[k][l]
        tfmatrix[k, currentword] = tfmatrix[k, currentword] + 1
        #tfmatrix[k, currentword] = tfmatrix[k, currentword]/len(mycrawled_prunedtexts[k])
        temp_dfvector[0, currentword] = 1
    # add which words occur in this document to overall document counts
    
    dfvector = dfvector + temp_dfvector

In [43]:
# length-normalized frequency for the TF part
for i in range(n_docs):
    for j in range(len(tfmatrix.data[i])):
        tfmatrix.data[i][j] = tfmatrix.data[i][j] / len(mycrawled_prunedtexts[i])

In [44]:
sum(tfmatrix.data[0]), sum(tfmatrix.data[1]) # sum to 1 because of the normalization

(1.0000000000000009, 0.9999999999999999)

In [45]:
sum(tfmatrix.data[2]), sum(tfmatrix.data[3])

(0.9999999999999994, 1.0000000000000002)

In [46]:
# create IDF and TF-IDF vectors
# smoothed logarithmic idf
idfvector = np.squeeze(np.array(dfvector.todense()))
idfvector = 1 + np.log(((idfvector + 1) ** -1) * n_docs)

In [47]:
idfvector

array([6.89615437, 7.68461173, 6.89615437, ..., 7.50229017, 6.99146455,
       7.34813949])

In [48]:
# use the count statistics to compute the tf-idf matrix
tfidfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab))
# use the count statistics to compute the tf-idf matrix
for k in range(n_docs):
    # find nonzero term frequencies
    tempindices = np.nonzero(tfmatrix[k, :])[1]
    tfterm = np.squeeze(np.array(tfmatrix[k, tempindices].todense()))
    tfidfmatrix[k, tempindices] = tfterm * idfvector[tempindices]

In [49]:
tfidfmatrix.shape

(4000, 12087)

##### 6.1d

In [50]:
# Compute 10 factors from LSA
n_low_dimensions = 10
Uleft, D, UrightT = scipy.sparse.linalg.svds(tfidfmatrix.todense(), k=n_low_dimensions)

In [51]:
# Examine the singular values
print(D)

[4.94213356 4.97446492 5.07663717 5.11207295 5.22913741 5.62844437
 5.93986337 6.73541547 7.36648552 7.78843854]


In [52]:
UrightT.shape 

(10, 12087)

In [53]:
print(UrightT) # 10 factors

[[-7.48652045e-05 -4.38742249e-05 -7.48652045e-05 ... -7.88168035e-05
   5.36025794e-04 -1.55600569e-04]
 [-5.34351494e-04 -3.38914712e-04 -5.34351494e-04 ... -7.30798271e-04
   3.80269118e-03 -8.52743341e-04]
 [ 8.93116267e-05  2.60207810e-05  8.93116267e-05 ...  7.54251556e-05
   2.12102348e-04  9.34886466e-05]
 ...
 [-7.34234699e-05 -3.95872799e-05 -7.34234699e-05 ... -8.23423235e-05
  -1.12980412e-04 -1.28019655e-04]
 [-3.63891590e-06 -1.67643667e-06 -3.63891590e-06 ... -2.45962949e-06
  -4.46098184e-06 -1.99512910e-06]
 [-5.13664759e-05 -1.81954707e-05 -5.13664759e-05 ... -6.57241349e-05
  -6.79921477e-05 -2.73291720e-05]]


In [54]:
# Examine a factor (here the one with largest singular value)
print(UrightT[9, :])

[-5.13664759e-05 -1.81954707e-05 -5.13664759e-05 ... -6.57241349e-05
 -6.79921477e-05 -2.73291720e-05]


##### 6.1e

In [55]:
# 10 words with largest absolute weights in each factor
for i in range(UrightT.shape[0]):
    print('10 words with largest absolute weights in factor ' + str(i+1))
    print('-------------------------------------------------------------')
    topweights_indices=np.argsort(-1*np.abs(UrightT[i,:]))
    print(remaining_vocabulary[topweights_indices[0:10]])
    print('\n')

10 words with largest absolute weights in factor 1
-------------------------------------------------------------
['osf.org' 'research.nj.nec.com' 'tin' 'x-newsreader' '1.1' 'group'
 'version' 'appreciate' 'pl8' 'honda']


10 words with largest absolute weights in factor 2
-------------------------------------------------------------
['tin' 'x-newsreader' '1.1' 'version' 'pl8' 'group' 'appreciate' 'jewish'
 'honda' 'pl4']


10 words with largest absolute weights in factor 3
-------------------------------------------------------------
['altima' 'generic' 'jewish' 'kingman' 'appreciate' 'stanza' 'nissan'
 'group' 'paperwork' 'nameplate']


10 words with largest absolute weights in factor 4
-------------------------------------------------------------
['jewish' 'kingman' 'tin' 'x-newsreader' '1.1' 'lafibm.lafayette.edu'
 'vb30' 'version' 'acad.drake.edu' 'altima']


10 words with largest absolute weights in factor 5
-------------------------------------------------------------
['appreciat

**The words in the factors seem related to the newsgroups**
- **honda, nissan and license seem related to rec.motorcycles newsgroup**
- **honda, stanza, nissan, altima, viper and lincense seem related to rec.autos newsgroup**
- **career, captain seem related to rec.sport.hockey newsgroup**
- **career, baseman, cub, yankee and captain seem related to rec.sport.baseball newsgroup**

##### 6.1f

In [56]:
# Compute 15 factors from LSA
n_low_dimensions = 15
Uleft, D, UrightT = scipy.sparse.linalg.svds(tfidfmatrix.todense(), k=n_low_dimensions)

In [57]:
# Examine the singular values
print(D) 

[4.47621528 4.61852057 4.71696298 4.75945967 4.78296889 4.94213356
 4.97446492 5.07663717 5.11207295 5.22913741 5.62844437 5.93986337
 6.73541547 7.36648552 7.78843854]


In [58]:
UrightT.shape # 15 factors

(15, 12087)

In [59]:
# 10 words with largest absolute weights in each factor
for i in range(UrightT.shape[0]):
    print('10 words with largest absolute weights in factor ' + str(i+1))
    print('-------------------------------------------------------------')
    topweights_indices=np.argsort(-1*np.abs(UrightT[i,:]))
    print(remaining_vocabulary[topweights_indices[0:10]])
    print('\n')

10 words with largest absolute weights in factor 1
-------------------------------------------------------------
['blah' 'yankee' 'eliot' 'stats' 'thank' 'arocha' 'group' 'jeffries'
 'status' 'dolven']


10 words with largest absolute weights in factor 2
-------------------------------------------------------------
['honda' 'subscribe' 'tour' 'group' 'lake' 'blah' 'forget' 'final'
 'advantage' 'stats']


10 words with largest absolute weights in factor 3
-------------------------------------------------------------
['group' 'lake' 'final' 'maine' 'advantage' '5-4' 'blah' 'lssu' 'terry'
 'appreciate']


10 words with largest absolute weights in factor 4
-------------------------------------------------------------
['advantage' 'lake' 'final' 'group' 'maine' '5-4' 'lssu' 'blah' 'terry'
 'appreciate']


10 words with largest absolute weights in factor 5
-------------------------------------------------------------
['group' 'advantage' 'lake' 'final' 'blah' 'honda' 'maine' 'subscribe'
 '5-

- **The last 10 factors produce the same words as the 10 factors in (e)**
- **The new 5 factors seem less meaningful as they have fewer words that seem related to the newsgroups**


##### Exercise 6.2

##### 6.2a

In [130]:
# create count vectors
n_docs = len(mycrawled_prunedtexts) # 4000=total number of documents
n_vocab = len(remaining_vocabulary) # 12087
# matrix of term frequencies
tfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab)) # 4000x12087
# row vector of document frequencies
dfvector = scipy.sparse.lil_matrix((1, n_vocab)) #1x12087

In [131]:
n_docs, n_vocab

(4000, 12087)

In [132]:
tfmatrix.shape, dfvector.shape

((4000, 12087), (1, 12087))

In [133]:
# loop over the documents
for k in range(n_docs):
    # row vector of which words occur in this document
    temp_dfvector = scipy.sparse.lil_matrix((1, n_vocab)) # #1x12087
    # we loop over the words in each document
    for l in range(len(mycrawled_prunedtexts[k])):
        # add current word to term-frequency count and document count
        currentword = myindices_in_prunedvocabulary[k][l]
        tfmatrix[k, currentword] = tfmatrix[k, currentword] + 1
        #tfmatrix[k, currentword] = tfmatrix[k, currentword]/len(mycrawled_prunedtexts[k])
        temp_dfvector[0, currentword] = 1
    # add which words occur in this document to overall document counts
    
    dfvector = dfvector + temp_dfvector

In [134]:
tfmatrix.shape

(4000, 12087)

##### 6.2b

In [135]:
def plsa(document_to_word_matrix, n_topics, n_iterations):
    n_docs=np.shape(document_to_word_matrix)[0] # Number of documents 
    n_vocab=np.shape(document_to_word_matrix)[1] # Number of vocabulary words
    theta = scipy.stats.uniform.rvs(size=(n_vocab, n_topics)) # Prob of words per topic: random init
    theta = theta/np.matlib.repmat(np.sum(theta,axis=0),n_vocab,1)
    psi = scipy.stats.uniform.rvs(size=(n_topics,n_docs)) # Probs topics per document: random init
    psi = psi/np.matlib.repmat(np.sum(psi,axis=0),n_topics,1)
    n_words_in_docs = np.squeeze(np.array(np.sum(document_to_word_matrix,axis=1))) # Numbers of words in documents: 
                                                                                   # computed once
    n_totalwords = np.sum(n_words_in_docs) # Total number of words: computed once
    pi = n_words_in_docs/n_totalwords # Document probs: computed once
    for myiter in range(n_iterations): # Perform Expectation-Maximization iterations
        # ===Perform E-step====
        doc_word_to_topics = [] # Compute theta_{v|t}psi_{t|d}/sum_t' theta_{v|t'}psi_{t'|d}
        doc_word_to_topic_sum = np.zeros((n_docs,n_vocab))
        for t in range(n_topics):
            doc_word_to_topict = np.matlib.repmat(theta[:,t], n_docs,1) * np.matlib.repmat(psi[t,:],n_vocab,1).T
            myepsilon=1e-14 # Add a positive number to avoid divisions by zero
            doc_word_to_topict += myepsilon
            doc_word_to_topics.append(doc_word_to_topict)
            doc_word_to_topic_sum += doc_word_to_topict
    for t in range(n_topics):
        doc_word_to_topics[t] /= doc_word_to_topic_sum
    # =======Perform M-step=======
    # Add a small number to word counts to avoid divisions by zero
    for t in range(n_topics): # Compute document-to-topic probabilities.
        psi[t,:] = np.squeeze(np.array(np.sum(np.multiply(
            document_to_word_matrix + myepsilon, doc_word_to_topics[t]),axis=1)))
    psi /= np.matlib.repmat(np.sum(psi,axis=0),n_topics,1)
    for t in range(n_topics): # Compute topic-to-word probabilities
        theta[:,t]= np.squeeze(np.array(np.sum(np.multiply(
            document_to_word_matrix,doc_word_to_topics[t]),axis=0).T))
    theta /= np.matlib.repmat(np.sum(theta,axis=0),n_vocab,1)
    return(pi,psi,theta)

In [136]:
# Run PLSA
n_topics = 10
n_iterations = 200
pi, psi, theta = plsa(tfmatrix.todense(), n_topics, n_iterations)

In [137]:
# Examine the factor probabilities p(t) = sum_d p(t|d) p(d)
print(np.sum(psi * np.matlib.repmat(pi, n_topics,1), axis=1))

[0.1022757  0.10237872 0.10360222 0.09754084 0.09814357 0.09818026
 0.09904054 0.10307408 0.09505071 0.10071336]


In [138]:
# Examine a factor (here the one with largest probability, factor 7)
print(theta[:, 7])

[4.54652329e-05 1.90239634e-05 4.62941613e-05 ... 1.16889231e-05
 6.51174676e-05 2.45828384e-05]


In [139]:
theta[:, 7].shape

(12087,)

In [141]:
psi[7,:].shape

(4000,)

In [142]:
np.matlib.repmat(pi, n_topics,1).shape

(10, 4000)

In [143]:
theta.shape

(12087, 10)

##### 6.2c

In [145]:
# 10 words with largest absolute weights in each factor
for i in range(theta.shape[1]):
    print('10 words with largest absolute weights in factor ' + str(i+1))
    print('-------------------------------------------------------------')
    topweights_indices = np.argsort(-1 * np.abs(theta[:, i]))
    print(remaining_vocabulary[topweights_indices[0:10]])
    print('\n')

10 words with largest absolute weights in factor 1
-------------------------------------------------------------
['ago' 'cost' 'final' 'det' 'hold' 'york' 'phone' 'beat' 'e-mail'
 'mention']


10 words with largest absolute weights in factor 2
-------------------------------------------------------------
['buffalo' 'york' 'job' 'allow' 'mind' 'bos' 'mention' 'pull' 'wait'
 'final']


10 words with largest absolute weights in factor 3
-------------------------------------------------------------
['later' 'group' 'forget' 'saw' 'mention' 'hope' 'month' 'cal'
 'disclaimer' 'late']


10 words with largest absolute weights in factor 4
-------------------------------------------------------------
['young' 'ice' 'almost' 'past' 'cost' 'local' 'insurance' 'bill' 'mind'
 'man']


10 words with largest absolute weights in factor 5
-------------------------------------------------------------
['large' 'beat' 'compare' 'friend' 'order' 'later' 'ago' 'forget' 'set'
 'tie']


10 words with largest a

**Some words in the factors seem related to the newsgroups**
- **final, beat, york, pull, ice, performance, penalty seem related to rec.sport.baseball and rec.sport.hockey newsgroups**
- **insurance, vehicle, ford, honda, rider seem related to rec.autos and rec.motorcycles newsgroups**

##### 6.2d

In [146]:
psi.shape

(10, 4000)

In [208]:
# 10 words with largest absolute weights in each factor
for i in range(psi.shape[0]):
    topweights_indices = np.argsort(-1 * np.abs(psi[i, :]))
    print('Document index with highest factor probability: ' + str(topweights_indices[0] + 1))
    print('-----------------------------------------------------------------------')
    print('First 100 words of factor ' + str(i+1) + ' of the document with highest probability')
    print('-----------------------------------------------------------------------')
    print(remaining_vocabulary[topweights_indices[0:100]])
    print('\n')

Document index with highest factor probability: 516
-----------------------------------------------------------------------
First 100 words of factor 1 of the document with highest probability
-----------------------------------------------------------------------
['0.565' '.20' '412' 'convenient' '*chain*' '6-4' 'ciccerelli' '02-00'
 'closer' "'62" '3com' 'cbs' '415' '1960' '1qkkodinn5f5' '0.62' 'buf/sd'
 '571' 'advice' 'continually' 'ash' 'author' '0.00' '-r-i-d-e-r'
 'compulsive' 'citadel' 'charles.a.rogers' 'blvd' '3732' '5-1' '175'
 '.signature' '.294' '984' 'buzz' '139' 'cod' '240+' '*young*' 'canucks'
 'barry' 'baumgartner' '20-' 'cabletron' '__/' 'bigot' '300' 'aptitude'
 'core' 'battle' '0.879' 'adolf' 'british' '227' '.405' '0.74' 'accident'
 '11-12' '36.4' '10.00' '0111' 'bielecki' 'advance' '.427' 'acquisition'
 'berube' '0.26' 'assure' '0.19' '1000km' '.274' 'category' '0.889'
 'cleaner-burning' 'brock' '1040' '4-6' 'bullet' 'bowman' '85.0' '328'
 '0.43' 'blumstein' 'bein'

***6.2e***

**Many words in each factor seem not to be meaningful and related to the newsgroups.**

#### Exercise 6.3

##### 6.3f

In [230]:
import gensim
gensim_docs = mycrawled_prunedtexts
gensim_dictionary = gensim.corpora.Dictionary(gensim_docs)
# Create the document-term vectors
gensim_docvectors = [gensim_dictionary.doc2bow(doc) for doc in gensim_docs]

##### 6.3g

In [241]:
# Run the LDA optimization for 10 topics
numtopics=10
randomseed=124574527
numiters=10000
ninits=10
gensim_ldamodel = gensim.models.ldamodel.LdaModel(gensim_docvectors, id2word=gensim_dictionary,
                                                 num_topics=numtopics, iterations=numiters, random_state=randomseed)

##### 6.3h

In [242]:
gensim_ldamodel.print_topics(num_topics=numtopics, num_words=10)

[(0,
  '0.007*"captain" + 0.006*"gant" + 0.005*"philadelphia" + 0.005*"hirschbeck" + 0.005*"puck" + 0.004*"devil" + 0.003*"umpire" + 0.003*"hawk" + 0.003*"suck" + 0.003*"strike"'),
 (1,
  '0.006*"championship" + 0.004*"maynard" + 0.004*"arena" + 0.004*"+/-" + 0.003*"draft" + 0.003*"station" + 0.003*"winner" + 0.003*"halifax" + 0.003*"pen" + 0.003*"paul"'),
 (2,
  '0.005*"puck" + 0.005*"buffalo" + 0.003*"total" + 0.003*"van" + 0.003*"career" + 0.003*"ott" + 0.003*"bond" + 0.002*"rochester" + 0.002*"compare" + 0.002*"morris"'),
 (3,
  '0.006*"det" + 0.005*"tor" + 0.005*"pit" + 0.005*"bos" + 0.005*"montreal" + 0.004*"cal" + 0.004*"min" + 0.003*"stl" + 0.003*"minnesota" + 0.003*"lock"'),
 (4,
  '0.008*"penalty" + 0.004*"conference" + 0.004*"quebec" + 0.004*"hartford" + 0.003*"montreal" + 0.003*"___" + 0.003*"ottawa" + 0.002*"sanderson" + 0.002*"tie" + 0.002*"louis"'),
 (5,
  '0.007*"stanley" + 0.004*"final" + 0.004*"joseph" + 0.003*"pen" + 0.003*"european" + 0.003*"minute" + 0.003*"jet" + 

- ***The top words in each topic seem related to sports - baseball and hockey***

##### 6.3i

In [243]:
# Get topic content: term-topic probabilities
gensim_termtopicprobabilities = gensim_ldamodel.get_topics()
# Get topic prevalences per document, and overall topic prevalences
# (expected amount of documents per topic)
overallstrengths = np.zeros((numtopics, 1))
documentstrengths = np.zeros((len(gensim_docvectors), numtopics)) # 4000x10
for k in range(len(gensim_docvectors)):
    topicstrengths = gensim_ldamodel.get_document_topics(gensim_docvectors[k], minimum_probability=0)
    for m in range(len(topicstrengths)):
        documentstrengths[k][topicstrengths[m][0]] = topicstrengths[m][1]
        overallstrengths[topicstrengths[m][0]] = overallstrengths[topicstrengths[m][0]] + topicstrengths[m][1]

In [244]:
for i in range(numtopics):
    word_list = []
    topweights_indices = np.argsort(-1 * np.abs(documentstrengths[:, i]))
    print('Document with highest topic probability: ' + str(topweights_indices[0] + 1))
    print('-----------------------------------------------------------------------')
    print('First 100 words of topic ' + str(i+1) + ' of the document with highest topic probability')
    print('-----------------------------------------------------------------------')
    words_with_probabilities = gensim_ldamodel.show_topic(i, topn=100)
    for word_with_probability in words_with_probabilities:
        word_list.append(word_with_probability)
    print(word_list)
    print('\n')

Document with highest topic probability: 1956
-----------------------------------------------------------------------
First 100 words of topic 1 of the document with highest topic probability
-----------------------------------------------------------------------
[('captain', 0.007223059), ('gant', 0.0064131706), ('philadelphia', 0.0046833158), ('hirschbeck', 0.0046017454), ('puck', 0.0045988844), ('devil', 0.0038172475), ('umpire', 0.003268002), ('hawk', 0.0031789003), ('suck', 0.003171802), ('strike', 0.0026171238), ('coverage', 0.002407789), ('hitter', 0.0023553455), ('helmet', 0.0023364287), ('order', 0.0023089394), ('stand', 0.0022562516), ('coach', 0.0022156944), ('left', 0.0021053758), ('slot', 0.002073364), ('ice', 0.0020159716), ('box', 0.0020067089), ('saw', 0.0019916652), ('tournament', 0.0019056029), ('insurance', 0.001871022), ("'91", 0.001848288), ('champ', 0.0018168534), ('ray', 0.0018124336), ('skate', 0.0018108978), ('type', 0.0018077022), ('adobe.com', 0.0017357153), 

***6.3j***

***The contents of individual topics seem meaningful and related to the newsgroups***

##### 6.3k

In [235]:
# Run the LDA optimization for 15 topics
numtopics=15
gensim_ldamodel = gensim.models.ldamodel.LdaModel(gensim_docvectors, id2word=gensim_dictionary,
                                                 num_topics=numtopics, iterations=numiters, random_state=randomseed)

In [236]:
gensim_ldamodel.print_topics(num_topics=numtopics, num_words=10)

[(0,
  '0.010*"captain" + 0.007*"hawk" + 0.005*"philadelphia" + 0.005*"suck" + 0.005*"0.00" + 0.004*"coach" + 0.003*"swedish" + 0.003*"cap" + 0.003*"former" + 0.003*"stand"'),
 (1,
  '0.008*"maynard" + 0.006*"championship" + 0.006*"scoring" + 0.005*"winner" + 0.005*"defense" + 0.005*"2nd" + 0.005*"coach" + 0.005*"draft" + 0.004*"yankee" + 0.004*"station"'),
 (2,
  '0.005*"buffalo" + 0.003*"rochester" + 0.003*"draft" + 0.003*"club" + 0.003*"cape" + 0.003*"pit" + 0.003*"inning" + 0.003*"total" + 0.003*"rule" + 0.003*"1992-93"'),
 (3,
  '0.006*"montreal" + 0.005*"roy" + 0.005*"1-0" + 0.004*"pit" + 0.004*"minnesota" + 0.004*"pen" + 0.004*"lock" + 0.004*"mattingly" + 0.003*"michael" + 0.003*"norway"'),
 (4,
  '0.006*"hartford" + 0.004*"ottawa" + 0.004*"___" + 0.003*"morris" + 0.002*"+/-" + 0.002*"nick" + 0.002*"canadian" + 0.002*"view" + 0.002*"rider" + 0.002*"mind"'),
 (5,
  '0.018*"penalty" + 0.005*"minute" + 0.004*"final" + 0.004*"=====" + 0.004*"joseph" + 0.004*"delete" + 0.003*"hull" +

***The first 10 topics are not the same as the 10 topics in (h)***
- ***There are only some words in the first 10 topics that appear in the 10 topics in (h). These words have different probabilities.***

In [237]:
# Get topic content: term-topic probabilities
gensim_termtopicprobabilities = gensim_ldamodel.get_topics()
# Get topic prevalences per document, and overall topic prevalences
# (expected amount of documents per topic)
overallstrengths = np.zeros((numtopics, 1))
documentstrengths = np.zeros((len(gensim_docvectors), numtopics)) # 4000x15
for k in range(len(gensim_docvectors)):
    topicstrengths = gensim_ldamodel.get_document_topics(gensim_docvectors[k], minimum_probability=0)
    for m in range(len(topicstrengths)):
        documentstrengths[k][topicstrengths[m][0]] = topicstrengths[m][1]
        overallstrengths[topicstrengths[m][0]] = overallstrengths[topicstrengths[m][0]] + topicstrengths[m][1]

In [238]:
documentstrengths.shape

(4000, 15)

In [240]:
for i in range(numtopics):
    word_list = []
    topweights_indices = np.argsort(-1 * np.abs(documentstrengths[:, i]))
    print('Document with highest topic probability: ' + str(topweights_indices[0] + 1))
    print('-----------------------------------------------------------------------')
    print('First 100 words of topic ' + str(i+1) + ' of the document with highest topic probability')
    print('-----------------------------------------------------------------------')
    words_with_probabilities = gensim_ldamodel.show_topic(i, topn=100)
    for word_with_probability in words_with_probabilities:
        word_list.append(word_with_probability)
    print(word_list)
    print('\n')

Document with highest topic probability: 1956
-----------------------------------------------------------------------
First 100 words of topic 1 of the document with highest topic probability
-----------------------------------------------------------------------
[('captain', 0.010319422), ('hawk', 0.0074584656), ('philadelphia', 0.005420894), ('suck', 0.004754035), ('0.00', 0.0045288955), ('coach', 0.0037435545), ('swedish', 0.0032953615), ('cap', 0.002971868), ('former', 0.0029717328), ('stand', 0.0029034365), ('face', 0.0028993487), ('champ', 0.002867784), ('devil', 0.0027952315), ('keith', 0.0027301796), ('hitter', 0.0027274876), ('pittsburg', 0.0024270937), ('green', 0.002407949), ('dean', 0.0024045585), ('hold', 0.0023849893), ('left', 0.0023147583), ('type', 0.0022901236), ('keller', 0.0022689837), ('tournament', 0.0022403714), ('ice', 0.002207734), ('insurance', 0.0021073793), ('kkeller', 0.0020703613), ('cookson', 0.0020533018), ("'91", 0.0020426563), ('minute', 0.0020410076),

***The contents of individual topics also seem meaningful and related to the newsgroups***
- **There are only some words in the first 10 topics that appear in the 10 topics in (i) but with different probabilities**

In [179]:
count = 0
for i in gensim_ldamodel[gensim_docvectors]:
    print('doc : ', count, i)
    count += 1

doc :  0 [(5, 0.98830456)]
doc :  1 [(9, 0.9895229)]
doc :  2 [(4, 0.9718132)]
doc :  3 [(0, 0.9249026)]
doc :  4 [(6, 0.5737663), (9, 0.36904272)]
doc :  5 [(5, 0.983315)]
doc :  6 [(5, 0.9653462)]
doc :  7 [(1, 0.2338177), (9, 0.7451031)]
doc :  8 [(0, 0.9762776)]
doc :  9 [(0, 0.11379922), (5, 0.8619305)]
doc :  10 [(6, 0.95708895)]
doc :  11 [(0, 0.9947614)]
doc :  12 [(3, 0.95902306)]
doc :  13 [(7, 0.93069583)]
doc :  14 [(2, 0.97349083)]
doc :  15 [(9, 0.98765194)]
doc :  16 [(9, 0.98233795)]
doc :  17 [(6, 0.98160684)]
doc :  18 [(0, 0.1173234), (3, 0.5384582), (6, 0.33018884)]
doc :  19 [(7, 0.07466916), (9, 0.91280365)]
doc :  20 [(3, 0.66835415), (4, 0.316235)]
doc :  21 [(9, 0.9718486)]
doc :  22 [(1, 0.5019001), (8, 0.4829784)]
doc :  23 [(0, 0.016679354), (1, 0.016684098), (2, 0.016678859), (3, 0.016677998), (4, 0.016680801), (5, 0.84988385), (6, 0.016678777), (7, 0.016679991), (8, 0.016678344), (9, 0.016677912)]
doc :  24 [(7, 0.7055096), (8, 0.2596766)]
doc :  25 [(7, 0

doc :  182 [(2, 0.25276855), (6, 0.72902906)]
doc :  183 [(0, 0.98040926)]
doc :  184 [(2, 0.28998083), (9, 0.69904417)]
doc :  185 [(0, 0.13705005), (2, 0.83089864)]
doc :  186 [(2, 0.26850635), (5, 0.058957055), (6, 0.65730363)]
doc :  187 [(2, 0.80422246), (3, 0.17836761)]
doc :  188 [(3, 0.100342065), (5, 0.88608426)]
doc :  189 [(1, 0.9399292)]
doc :  190 [(0, 0.93560284)]
doc :  191 [(3, 0.9355174)]
doc :  192 [(2, 0.94703543)]
doc :  193 [(0, 0.014292673), (1, 0.8713676), (2, 0.014292056), (3, 0.014292031), (4, 0.014291986), (5, 0.014292462), (6, 0.014291981), (7, 0.0142935915), (8, 0.014292867), (9, 0.014292775)]
doc :  194 [(4, 0.9803986)]
doc :  195 [(2, 0.9590734)]
doc :  196 [(6, 0.95905554)]
doc :  197 [(2, 0.20908853), (4, 0.77858496)]
doc :  198 [(1, 0.28186873), (2, 0.09672869), (4, 0.6121776)]
doc :  199 [(2, 0.31310168), (8, 0.6835747)]
doc :  200 [(2, 0.99180895)]
doc :  201 [(0, 0.54165894), (2, 0.426307)]
doc :  202 [(9, 0.99020594)]
doc :  203 [(0, 0.21022977), (6

doc :  349 [(6, 0.9718483)]
doc :  350 [(9, 0.9436689)]
doc :  351 [(5, 0.218711), (9, 0.7492545)]
doc :  352 [(0, 0.16074064), (5, 0.82872236)]
doc :  353 [(0, 0.6397436), (9, 0.33916166)]
doc :  354 [(0, 0.95707875)]
doc :  355 [(4, 0.46284983), (7, 0.5266084)]
doc :  356 [(0, 0.984187)]
doc :  357 [(5, 0.98952335)]
doc :  358 [(1, 0.9470198)]
doc :  359 [(1, 0.9399547)]
doc :  360 [(9, 0.98121375)]
doc :  361 [(0, 0.050015714), (1, 0.050009314), (2, 0.05001022), (3, 0.050014608), (4, 0.05002104), (5, 0.54987156), (6, 0.050017428), (7, 0.050013617), (8, 0.05001325), (9, 0.05001327)]
doc :  362 [(5, 0.9624544)]
doc :  363 [(2, 0.9608015)]
doc :  364 [(1, 0.12546076), (9, 0.8554644)]
doc :  365 [(0, 0.99568677)]
doc :  366 [(6, 0.99030775)]
doc :  367 [(6, 0.58138037), (8, 0.41008878)]
doc :  368 [(3, 0.98546344)]
doc :  369 [(8, 0.9470167)]
doc :  370 [(3, 0.4332526), (5, 0.5196115)]
doc :  371 [(6, 0.99785984)]
doc :  372 [(1, 0.9762974)]
doc :  373 [(1, 0.07720679), (2, 0.9022526)]


doc :  565 [(1, 0.52705866), (7, 0.46002153)]
doc :  566 [(4, 0.9749758)]
doc :  567 [(0, 0.95905226)]
doc :  568 [(4, 0.9040007), (5, 0.07895128)]
doc :  569 [(0, 0.34578145), (1, 0.12988442), (3, 0.15442888), (4, 0.35364175)]
doc :  570 [(9, 0.98781246)]
doc :  571 [(0, 0.6408236), (6, 0.33296478), (8, 0.021628242)]
doc :  572 [(5, 0.123455435), (7, 0.8569998)]
doc :  573 [(4, 0.5823448), (8, 0.1642585), (9, 0.2183644)]
doc :  574 [(0, 0.17863294), (2, 0.21066716), (7, 0.25785607), (9, 0.3305805)]
doc :  575 [(4, 0.94368917)]
doc :  576 [(6, 0.3451248), (7, 0.6388581)]
doc :  577 [(7, 0.9902089)]
doc :  578 [(6, 0.8934515), (9, 0.09913154)]
doc :  579 [(0, 0.17368911), (9, 0.8120078)]
doc :  580 [(4, 0.9921639)]
doc :  581 [(1, 0.98766387)]
doc :  582 [(6, 0.98712754)]
doc :  583 [(1, 0.7707895), (5, 0.068515345), (8, 0.15193468)]
doc :  584 [(1, 0.19699293), (2, 0.755909)]
doc :  585 [(4, 0.9608199)]
doc :  586 [(1, 0.19900398), (5, 0.060854096), (9, 0.71675897)]
doc :  587 [(3, 0.9

doc :  771 [(3, 0.19971596), (4, 0.09769519), (6, 0.24029335), (7, 0.45459378)]
doc :  772 [(4, 0.7035156), (5, 0.2905486)]
doc :  773 [(5, 0.9871286)]
doc :  774 [(9, 0.98653334)]
doc :  775 [(5, 0.4788906), (7, 0.48773548)]
doc :  776 [(0, 0.7011972), (8, 0.2679898)]
doc :  777 [(8, 0.04574805), (9, 0.9454498)]
doc :  778 [(5, 0.9624583)]
doc :  779 [(7, 0.9922327)]
doc :  780 [(3, 0.08087746), (6, 0.90243334)]
doc :  781 [(0, 0.371109), (4, 0.60386264)]
doc :  782 [(0, 0.6735145), (6, 0.30645502)]
doc :  783 [(4, 0.34311053), (7, 0.6373527)]
doc :  784 [(3, 0.06414598), (6, 0.92293286)]
doc :  785 [(2, 0.6346859), (4, 0.35476944)]
doc :  786 [(7, 0.17998214), (8, 0.8111097)]
doc :  787 [(6, 0.99296135)]
doc :  788 [(5, 0.9830003)]
doc :  789 [(2, 0.64241487), (8, 0.3353324)]
doc :  790 [(3, 0.9178289), (7, 0.072856575)]
doc :  791 [(0, 0.28143212), (8, 0.71094114)]
doc :  792 [(1, 0.1325604), (5, 0.13500132), (9, 0.7054335)]
doc :  793 [(9, 0.9876578)]
doc :  794 [(5, 0.38700348), (

...

doc :  3518 [(4, 0.6125364), (5, 0.3442705)]
doc :  3519 [(3, 0.076270275), (6, 0.7259916), (8, 0.16419017)]
doc :  3520 [(4, 0.47041306), (5, 0.10999874), (8, 0.19708797), (9, 0.21064222)]
doc :  3521 [(1, 0.279532), (4, 0.42166284), (7, 0.17084658), (8, 0.096451074)]
doc :  3522 [(0, 0.079251096), (3, 0.2108574), (4, 0.22775906), (6, 0.45646688)]
doc :  3523 [(2, 0.21566379), (3, 0.5196524), (4, 0.10272379), (7, 0.06437056), (8, 0.020350497), (9, 0.07479014)]
doc :  3524 [(0, 0.012505678), (1, 0.88744354), (2, 0.012505711), (3, 0.012507266), (4, 0.0125059085), (5, 0.012507293), (6, 0.012509762), (7, 0.012503871), (8, 0.012505796), (9, 0.012505206)]
doc :  3525 [(2, 0.44229573), (3, 0.27796736), (5, 0.1430803), (8, 0.11721313)]
doc :  3526 [(0, 0.4343096), (7, 0.49935418)]
doc :  3527 [(0, 0.40650952), (8, 0.2759506), (9, 0.2808258)]
doc :  3528 [(0, 0.34343717), (2, 0.103778675), (3, 0.49528962)]
doc :  3529 [(2, 0.19137236), (3, 0.73216194), (9, 0.058667578)]
doc :  3530 [(1, 0.2107

doc :  3628 [(1, 0.05964611), (2, 0.2130851), (4, 0.12595639), (5, 0.02551942), (7, 0.18361498), (9, 0.3875639)]
doc :  3629 [(2, 0.13249835), (3, 0.03478205), (4, 0.0813935), (5, 0.20195211), (6, 0.16843869), (7, 0.03560127), (8, 0.11380905), (9, 0.22760488)]
doc :  3630 [(2, 0.11062948), (7, 0.40062463), (9, 0.465749)]
doc :  3631 [(7, 0.20701002), (8, 0.29801375), (9, 0.4821813)]
doc :  3632 [(0, 0.276104), (2, 0.28584006), (4, 0.10385409), (5, 0.05397507), (8, 0.22535224), (9, 0.044849224)]
doc :  3633 [(0, 0.084666155), (1, 0.35029215), (4, 0.30763462), (7, 0.23241946)]
doc :  3634 [(0, 0.1612364), (1, 0.010531208), (2, 0.010528611), (3, 0.010527956), (4, 0.27203843), (5, 0.4930217), (6, 0.010529662), (7, 0.010527454), (8, 0.010529803), (9, 0.010528812)]
doc :  3635 [(1, 0.3184992), (4, 0.1947696), (7, 0.1695822), (9, 0.29163545)]
doc :  3636 [(0, 0.2258889), (2, 0.25332204), (3, 0.21324266), (5, 0.28252092)]
doc :  3637 [(0, 0.38627484), (6, 0.58764464)]
doc :  3638 [(0, 0.565913

doc :  3730 [(4, 0.3435393), (7, 0.25280207), (9, 0.37625316)]
doc :  3731 [(4, 0.6621877), (9, 0.29674518)]
doc :  3732 [(1, 0.24157092), (4, 0.42432007), (5, 0.17531668), (6, 0.1351672)]
doc :  3733 [(0, 0.05444838), (4, 0.14441794), (8, 0.15146777), (9, 0.63309366)]
doc :  3734 [(2, 0.3867087), (7, 0.0822068), (9, 0.5019433)]
doc :  3735 [(1, 0.359937), (3, 0.1563264), (5, 0.27264136), (9, 0.17968637)]
doc :  3736 [(1, 0.495563), (3, 0.18905866), (6, 0.09652286), (8, 0.20682621)]
doc :  3737 [(0, 0.6292142), (7, 0.32342193)]
doc :  3738 [(0, 0.14156212), (1, 0.040690966), (3, 0.33005038), (8, 0.17145982), (9, 0.3087773)]
doc :  3739 [(0, 0.03864288), (2, 0.33911422), (3, 0.030513637), (4, 0.21337801), (5, 0.074111454), (7, 0.02907684), (9, 0.27136704)]
doc :  3740 [(2, 0.10864311), (3, 0.6022147), (4, 0.034234412), (6, 0.02872102), (7, 0.1987534), (9, 0.022126257)]
doc :  3741 [(0, 0.37849614), (2, 0.16056933), (7, 0.23633729), (8, 0.19260883)]
doc :  3742 [(1, 0.37241915), (6, 0.23

doc :  3844 [(0, 0.4027255), (2, 0.37183762), (3, 0.20912158)]
doc :  3845 [(1, 0.67589116), (4, 0.078177035), (9, 0.2190893)]
doc :  3846 [(1, 0.43422344), (4, 0.40614507), (8, 0.12152239)]
doc :  3847 [(0, 0.5566607), (1, 0.16678885), (5, 0.2509451)]
doc :  3848 [(2, 0.21502472), (3, 0.13124295), (4, 0.3053413), (8, 0.15155546), (9, 0.1850839)]
doc :  3849 [(1, 0.37709755), (4, 0.46776986), (9, 0.13160734)]
doc :  3850 [(1, 0.16305417), (2, 0.33680704), (6, 0.36488554), (7, 0.05017426), (9, 0.070932224)]
doc :  3851 [(4, 0.026426807), (6, 0.07297011), (8, 0.04500817), (9, 0.84937847)]
doc :  3852 [(2, 0.34712943), (3, 0.29364595), (5, 0.31985185)]
doc :  3853 [(0, 0.1295524), (2, 0.31539837), (4, 0.46916083), (6, 0.06149338)]
doc :  3854 [(0, 0.18500821), (2, 0.25466943), (4, 0.38645566), (5, 0.1289887), (6, 0.035559647)]
doc :  3855 [(1, 0.018507997), (2, 0.10804754), (3, 0.3972863), (4, 0.047494404), (6, 0.0132140815), (7, 0.12603548), (9, 0.28792766)]
doc :  3856 [(0, 0.08072931),

doc :  3958 [(0, 0.06392269), (1, 0.24998434), (2, 0.3553172), (3, 0.101098955), (4, 0.14422427), (9, 0.08052529)]
doc :  3959 [(2, 0.5692711), (9, 0.39087442)]
doc :  3960 [(1, 0.30505943), (2, 0.20082031), (4, 0.4679809)]
doc :  3961 [(2, 0.25730518), (4, 0.06609933), (5, 0.6064897), (6, 0.03065339), (8, 0.03065982)]
doc :  3962 [(2, 0.08161634), (3, 0.1882696), (5, 0.06630047), (7, 0.4474775), (9, 0.20393683)]
doc :  3963 [(0, 0.06405211), (5, 0.6832235), (7, 0.08339469), (8, 0.15887727)]
doc :  3964 [(2, 0.14608604), (3, 0.14461143), (4, 0.07045235), (5, 0.20697899), (7, 0.4101368)]
doc :  3965 [(0, 0.015253053), (1, 0.015247951), (2, 0.23408501), (3, 0.015248676), (4, 0.015250681), (5, 0.015253185), (6, 0.0152489375), (7, 0.015247999), (8, 0.6439114), (9, 0.015253058)]
doc :  3966 [(1, 0.34348413), (2, 0.120468155), (7, 0.48783913)]
doc :  3967 [(0, 0.13589558), (1, 0.7053759), (2, 0.06470263), (6, 0.08626524)]
doc :  3968 [(3, 0.20565182), (7, 0.04035053), (8, 0.7357588)]
doc :  